In [1]:
from pydantic import BaseModel, Field
from typing import List
import guidance
import os


In [2]:
# # vllm hosted model using LiteLLM

# litellm_desc = {
#     "model_name": "Qwen/Qwen3-30B-A3B-Instruct-2507",
#     "litellm_params": {  # params for litellm completion/embedding call
#         "model": "hosted_vllm/Qwen/Qwen3-30B-A3B-Instruct-2507",
#         "api_key": os.environ.get("VLLM_API_KEY", "NO_KEY"), # set your vLLM API key if needed
#         "api_base": "http://localhost:8000/v1", # change to your vLLM API base URL
#     },
# }
# base_lm = guidance.models.experimental.LiteLLM(model_description=litellm_desc, echo=True)

In [2]:
base_lm = guidance.models.experimental.SglangModel(model="Qwen/Qwen3-30B-A3B-Instruct-2507", base_url="http://127.0.0.1:30000/v1" ,echo=True, api_key = os.environ.get("OPENAI_API_KEY", "NO_KEY"))

In [3]:
# def run_gen_test(lm):
#     with guidance.user():
#         lm += "What is the capital of France? and its population?"
#         lm += "Format your answer as follows: Capital: <capital>, Population: <population>"

#     with guidance.assistant():
#         lm += guidance.gen(max_tokens=1024, temperature=0.7, name="answer")
#         print(lm["answer"])

# run_gen_test(base_lm)

In [4]:
from pydantic import BaseModel, Field
from typing import List
import guidance
from guidance import models, system, user, assistant, json as gen_json, select
import json
from enum import Enum


class FirstPrinciples(BaseModel):
    absolute_truths: List[str] = Field(
        ...,
        description="Look at the problem statement and list the things that you know that are absolute truths."
    )


class GroundUpReasoning(BaseModel):
    """Model for the LLM-generated reasoning plan."""
    reason: str = Field(
        ...,
        description="Based on the known truths reason in a concise sharp manner from ground up to address the problem."
    )

class Reflection(BaseModel):
    """Model for an individual reasoning step execution."""
    evaluation: str = Field(
        ...,
        description="Reflect on your reasoning and identify gaps / uncertainties / mistakes."
    )

class ReasoningConclusion(BaseModel):
    """Model for the final reasoning conclusion."""
    conclusion: str = Field(
        ...,
        description="Address the identified gaps in the previous step if any and deduce the final conclusion."
    )

def dynamic_final_answer_schema(Options):
    class FinalAnswer(BaseModel):
        answer: Options = Field(..., description="Final answer.")

    return FinalAnswer


@guidance()
def _first_principles(llm):
    with assistant():
        llm += gen_json(
            name="first_principles",
            schema=FirstPrinciples,
            max_tokens=5000
        )
    return llm

@guidance()
def _generate_groundup_reasoning(llm):
    """
    Generate a custom reasoning plan tailored to the query.
    """
    with assistant():
        llm += gen_json(
            name="groundup_reasoning",
            schema=GroundUpReasoning,
            max_tokens=5000  # Reduced to encourage conciseness
        )
    return llm

@guidance()
def _reflection(llm):
    with assistant():
        llm += gen_json(
            name="reflection",
            schema=Reflection,
            max_tokens=5000
        )
    return llm
    

@guidance()
def _reasoning_conclusion(llm):
    with user():
        llm += """Based on the information, make logical deductions."""
    
    with assistant():
        llm += gen_json(
            name="reasoning_conclusion",
            schema=ReasoningConclusion,
            max_tokens=5000
        )

    return llm

@guidance
def dynamic_structured_reasoning(llm, query, output_choices):
    Options = Enum("Choices", {choice: choice for choice in output_choices})
    final_answer_schema = dynamic_final_answer_schema(Options)

    with system():
        llm += """You are an expert in problem solving. For any query, reason from first principles to solve the query. 
        Structure all outputs as JSON. Be very brief in your steps, tending towards concise but complete responses.
        """

    with user():
        llm += f"""Query: {query}
        Look at the problem statement and list the things that you know that are absolute truths -
        """

    llm += _first_principles()

    with user():
        llm += f"""
        Based on the known truths reason from ground up to address the problem, be extremely brief and to the point in your reasoning -
        """

    llm += _generate_groundup_reasoning()

    with user():
        llm += f"""
        Reflect to identify gaps / uncertainties / mistakes in reasoning process -
        """

    llm += _reflection()

    with user():
        llm += f"""
        Make logical deductions based on the information you have to arrive at a conclusion -
        """

    llm += _reasoning_conclusion()

    with user():
        llm += f"""
        Your final answer -
        """

    with assistant():
        llm += gen_json(
            name="answer",
            schema=final_answer_schema,
            max_tokens=1000)

    return llm

In [5]:
def generate(query, output_choices, lm = None):
    lm = lm or base_lm
    result_lm = lm + dynamic_structured_reasoning(
                    query=query,
                    output_choices = output_choices
                )

    answer = json.loads(result_lm.get("answer"))['answer']
    
    groundup_reasoning = json.loads(result_lm.get("groundup_reasoning"))['reason']
    first_principles = json.loads(result_lm.get("first_principles"))['absolute_truths']
    reflection = json.loads(result_lm.get("reflection"))['evaluation']   
    reasoning_conclusion = json.loads(result_lm.get("reasoning_conclusion"))['conclusion']
    return first_principles, groundup_reasoning, reflection, reasoning_conclusion, answer


In [6]:
from datasets import load_dataset

# Load the BBH dataset from Hugging Face
dataset = load_dataset("WildEval/ZebraLogic", 'mc_mode')



In [7]:
from json import JSONDecodeError


def evaluate_reasoning(row):
    """
    Placeholder function to evaluate your reasoning system on a single row.
    Replace with your dynamic_structured_reasoning function.
    """
    
    # from guidance.models.experimental import SglangModel
    # lm = SglangModel(
    #     model="Qwen/Qwen3-30B-A3B-Instruct-2507",
    #     base_url="http://127.0.0.1:30000/v1",
    #     echo=True,
    #     api_key=os.environ.get("OPENAI_API_KEY", "NO_KEY"),
    # )
    puzzle = row['puzzle']
    question = row['question']
    query = puzzle + '\n\n' + question
    choices = row['choices']
    target = row['answer']
    print(row['id'])
    # Your reasoning system should return a predicted answer
    try:
        first_principles, groundup_reasoning, reflection, reasoning_conclusion, pred = generate(query, choices)
    except JSONDecodeError as e:
        first_principles, groundup_reasoning, reflection, reasoning_conclusion, pred = ["JSONDecodeError"]*5
    return {
        'correct': pred == target,
        'first_principles': first_principles,
        'groundup_reasoning': groundup_reasoning,
        'reflection': reflection,
        'reasoning_conclusion': reasoning_conclusion,
        'predicted_answer': pred, 
    }


In [8]:
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import pandas as pd

# # df is already df.iloc[600:650]  (keeps original index)
# def run_parallel_preserve_index(df, workers=16):
#     idx_to_row = df.to_dict(orient="index")  # {index: rowdict}
#     results = {}

#     with ThreadPoolExecutor(max_workers=workers) as ex:
#         futs = {ex.submit(evaluate_reasoning, row): idx
#                 for idx, row in idx_to_row.items()}
#         for fut in as_completed(futs):
#             idx = futs[fut]
#             results[idx] = fut.result()

#     res_df = pd.DataFrame.from_dict(results, orient="index")
#     # ensure order matches df (and keep original index)
#     res_df = res_df.reindex(df.index)
#     return df.join(res_df)

# df = pd.DataFrame(dataset['test'])
# # df.head()
# # CURRENTLY FINISHED - [:600]

# df = df.iloc[600:605]
# df = run_parallel_preserve_index(df, workers=16)


In [ ]:
import pandas as pd

df = pd.DataFrame(dataset['test'])
# df.head()
# CURRENTLY FINISHED - [:1150]

df = df.iloc[1150:1200]

res = df.apply(lambda r: pd.Series(evaluate_reasoning(r)), axis=1)
df = pd.concat([df, res], axis=1)

lgp-test-2x5-0#mc-1


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

lgp-test-6x5-23#mc-15
lgp-test-3x3-13#mc-2
lgp-test-3x5-8#mc-10
lgp-test-5x4-5#mc-9
lgp-test-5x5-2#mc-24
lgp-test-3x3-10#mc-5
lgp-test-6x5-19#mc-21
lgp-test-4x2-37#mc-2
lgp-test-6x5-39#mc-21
lgp-test-5x5-31#mc-19
lgp-test-6x6-11#mc-17
lgp-test-6x6-28#mc-28
lgp-test-2x5-31#mc-9
lgp-test-4x4-12#mc-0
lgp-test-2x6-17#mc-5
lgp-test-6x4-9#mc-9
lgp-test-5x4-37#mc-1
lgp-test-5x6-8#mc-1
lgp-test-5x5-34#mc-12
lgp-test-4x6-38#mc-17
lgp-test-3x4-37#mc-1
lgp-test-2x6-32#mc-7
lgp-test-3x5-5#mc-9
lgp-test-2x5-17#mc-9
lgp-test-2x3-29#mc-5
lgp-test-4x6-3#mc-21
lgp-test-3x5-35#mc-9
lgp-test-4x6-16#mc-22
lgp-test-3x2-16#mc-5
lgp-test-2x6-28#mc-10
lgp-test-2x5-2#mc-3
lgp-test-3x6-23#mc-10
lgp-test-4x6-25#mc-2
lgp-test-5x6-6#mc-20
lgp-test-5x4-33#mc-9
lgp-test-4x4-3#mc-3
lgp-test-5x4-33#mc-2
lgp-test-5x5-36#mc-2
lgp-test-6x6-14#mc-33
lgp-test-3x2-3#mc-3
lgp-test-6x2-18#mc-9
lgp-test-3x4-13#mc-7
lgp-test-5x6-0#mc-7
lgp-test-2x4-11#mc-4
lgp-test-6x4-5#mc-8
lgp-test-3x3-30#mc-5
lgp-test-4x6-3#mc-22
lgp-test-2

In [16]:
df

,id,puzzle,question,choices,answer,created_at,correct,first_principles,groundup_reasoning,reflection,reasoning_conclusion,predicted_answer
1100,lgp-test-2x5-0#mc-1,"There are 2 houses, numbered 1 to 2 from left ...",What is BookGenre of the person who lives in H...,"[mystery, science fiction]",mystery,2024-07-03T21:21:31.302010,True,[House 2 has the person who loves science fict...,House 2 has science fiction (clue 2); Arnold i...,All steps follow logically from clues. No gaps...,"House 1: Eric, mystery, April, yellow, very sh...",mystery
1101,lgp-test-6x5-23#mc-15,"There are 6 houses, numbered 1 to 6 from left ...",What is Name of the person who lives in House 4?,"[Eric, Carol, Bob, Peter, Arnold, Alice]",Peter,2024-07-03T21:21:31.380572,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError
1102,lgp-test-3x3-13#mc-2,"There are 3 houses, numbered 1 to 3 from left ...",What is Smoothie of the person who lives in Ho...,"[watermelon, desert, cherry]",watermelon,2024-07-03T21:21:31.308077,True,"[Peter is the parent of Fred., Fred's parent i...","From clues: Eric likes Cherry, and is directly...",All steps follow logically from absolute truth...,The smoothie of the person in House 1 is Water...,watermelon
1103,lgp-test-3x5-8#mc-10,"There are 3 houses, numbered 1 to 3 from left ...",What is Name of the person who lives in House 3?,"[Arnold, Peter, Eric]",Peter,2024-07-03T21:21:31.312193,True,"[The person who loves pizza is Eric., The pers...",Eric is in House 1 (clue 3). Eric loves pizza ...,Initial reasoning correctly places Eric in Hou...,The person in House 3 is Peter.,Peter
1104,lgp-test-5x4-5#mc-9,"There are 5 houses, numbered 1 to 5 from left ...",What is Animal of the person who lives in Hous...,"[bird, fish, cat, dog, horse]",cat,2024-07-03T21:21:31.343800,False,[House 3 has the person named Peter (Clue 14)....,"House 3: Peter (Clue 14), Carnations (Clue 12)...",Step 1: Assigned horse to house 3 based on 'ho...,"House 3: Peter (Clue 14), Carnations (Clue 12)...",horse
1105,lgp-test-5x5-2#mc-24,"There are 5 houses, numbered 1 to 5 from left ...",What is Animal of the person who lives in Hous...,"[bird, dog, cat, horse, fish]",dog,2024-07-03T21:21:31.348335,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError
1106,lgp-test-3x3-10#mc-5,"There are 3 houses, numbered 1 to 3 from left ...",What is Hobby of the person who lives in House 2?,"[gardening, cooking, photography]",cooking,2024-07-03T21:21:31.307964,True,"[House 2 has the tea drinker., Peter is the te...",Peter is in House 2 (from clues 1 and 2). Arno...,Reasoning is logically consistent: Peter in Ho...,The hobby of the person in House 2 is cooking.,cooking
1107,lgp-test-6x5-19#mc-21,"There are 6 houses, numbered 1 to 6 from left ...",What is FavoriteSport of the person who lives ...,"[basketball, volleyball, swimming, baseball, s...",basketball,2024-07-03T21:21:31.379768,False,"[House 3: Peter (from clue 10), House 3: Lunch...","House 5 has white color (clue 3), so red hair ...",The chain volleyball → baseball → brown hair →...,The person in House 5 has baseball as their fa...,baseball
1108,lgp-test-4x2-37#mc-2,"There are 4 houses, numbered 1 to 4 from left ...",What is Name of the person who lives in House 2?,"[Peter, Eric, Arnold, Alice]",Peter,2024-07-03T21:21:31.319897,True,"[Eric's birthday is in September., The person ...",From Clues 1 and 2: Eric is in House 4. From C...,All steps follow logically from the clues. No ...,The person in House 2 is Peter.,Peter
1109,lgp-test-6x5-39#mc-21,"There are 6 houses, numbered 1 to 6 from left ...",What is BookGenre of the person who lives in H...,"[romance, fantasy, mystery, historical fiction...",romance,2024-07-03T21:21:31.384048,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError


In [17]:
df['correct'].value_counts()

correct
True     34
False    16
Name: count, dtype: int64

In [18]:
path = './zebralogic_eval.csv'
df.to_csv(path, mode="a", index=False, header=not os.path.exists(path))

In [22]:
dub = pd.read_csv(path)


In [23]:
dub['correct'].value_counts()

correct
True     850
False    300
Name: count, dtype: int64